# **Configuração Geral**

## Imports

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from fractions import Fraction
import joblib
import scipy.stats as stats

# Machine Learning Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

## Funções

In [43]:
# Converte uma string que representa polegadas para um float
def polegadas(value):

    # Divide a string recebida
    value = value.strip()

    # Se a string tem mais de uma parte, obtém a parte inteira e calcula a parte decimal
    if ' ' in value:
        inteiro, decimal = value.split(' ') # a parte inteira é o que está antes do ' ', a parte decimal é o que está depois
        polegadas = float(inteiro) + float(Fraction(decimal))

    else:
        polegadas = float(value)
    
    return polegadas

# -------------------------------
# Testa, otimiza e retorna o modelo de melhor performance
def seleciona_modelos(df, model_param_grid):
    # Preparar os dados
    df1 = df.copy()
    df_train = df1[df1['tipo_poco'] != 'DIRECIONAL']
    
    X_train = df_train[['fase', 'lda_lamina_d_agua', 'metragem', 'nfases', 'diametro_fase_polegadas']]
    y_train = df_train['tipo_poco']
    
    # Codificar target
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    
    best_model = None
    best_score = 0
    best_metrics = None
    
    # Iterar sobre os modelos e grids de parâmetros fornecidos
    for model_name, (model, param_grid) in model_param_grid.items():
        print(f"\nTestando {model_name}...")
        
        # Inicializar GridSearchCV
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train_encoded)
        
        # Obter o melhor modelo
        current_best_model = grid_search.best_estimator_
        current_best_score = grid_search.best_score_
        
        # Avaliar o modelo no conjunto de treinamento completo
        y_train_pred_encoded = current_best_model.predict(X_train)
        
        # Imprimir relatórios de avaliação
        # print(f"Modelo: {model_name}")
        # print("Melhores Parâmetros Encontrados:")
        # print(grid_search.best_params_)
        
        # print("Relatório de Classificação (Treinamento):")
        report = classification_report(y_train_encoded, y_train_pred_encoded, target_names=label_encoder.classes_)
        # print(report)
        
        # print("Acurácia no Conjunto de Treinamento:")
        accuracy = accuracy_score(y_train_encoded, y_train_pred_encoded)
        # print(f"{accuracy:.4f}")
        
        # Verificar se é o melhor modelo até agora
        if current_best_score > best_score:
            best_score = current_best_score
            best_model = current_best_model
            best_metrics = {
                'model_name': model_name,
                'best_params': grid_search.best_params_,
                'classification_report': report,
                'accuracy': accuracy
            }
    
    # Imprimir as métricas do melhor modelo dentre todos os testados
    print("\nMelhor Modelo:")
    print(f"Modelo: {best_metrics['model_name']}")
    print("Melhores Parâmetros Encontrados:")
    print(best_metrics['best_params'])
    
    print("Relatório de Classificação (Treinamento):")
    print(best_metrics['classification_report'])
    
    print("Acurácia no Conjunto de Treinamento:")
    print(f"{best_metrics['accuracy']:.4f}")
    
    return best_model
# -------------------------------

# Função para salvar a grid de gráficos
def save_grid_plot(filename):
    plt.savefig(f'img/{filename}')
    plt.close()


# -------------------------------
# Função para gerar e salvar gráficos de distribuição em uma grid
def plot_distributions_grid(df, columns):
    num_vars = len(columns)
    fig, axes = plt.subplots(nrows=(num_vars + 2) // 3, ncols=3, figsize=(18, 4 * ((num_vars + 2) // 3)))
    axes = axes.flatten()
    
    for i, column in enumerate(columns):
        sns.histplot(df[column], kde=True, ax=axes[i])
        axes[i].set_title(f'Distribuição de {column}')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')  # Desliga os eixos restantes
    
    plt.tight_layout()
    save_grid_plot('distributions_grid.png')

# **Carregando Dados**

In [29]:
# Carregando dados csv
df = pd.read_csv('data/dados_pocos.csv')
df.head()

,Unnamed: 0,Nome do Poço,Fase,Tipo Poço,LDA (Lâmina D'Água),Diâmetro Fase,Metragem,NFases
0,0,6C5G4M,1,VERTICAL,598.0,26,607.0,4
1,1,6C5G4M,2,VERTICAL,598.0,17 1/2,1306.0,4
2,2,6C5G4M,3,VERTICAL,598.0,12 1/4,1201.0,4
3,3,6C5G4M,4,VERTICAL,598.0,8 1/2,1445.0,4
4,4,8030JC,1,VERTICAL,0.0,17 1/2,410.0,3


# **Limpeza dos Dados**
- Tratamento de valores ausentes
- Remoção de outliers
- Correção de tipos de dados

In [30]:
# Renomear as colunas para o padrão PEP8
df.columns = ['index', 'nome_do_poco', 'fase', 'tipo_poco', 'lda_lamina_d_agua', 'diametro_fase', 'metragem', 'nfases']

# Remover a coluna de índices 'Unnamed: 0'
df = df.drop(columns=['index'])

# Exibir df
df.head()

,nome_do_poco,fase,tipo_poco,lda_lamina_d_agua,diametro_fase,metragem,nfases
0,6C5G4M,1,VERTICAL,598.0,26,607.0,4
1,6C5G4M,2,VERTICAL,598.0,17 1/2,1306.0,4
2,6C5G4M,3,VERTICAL,598.0,12 1/4,1201.0,4
3,6C5G4M,4,VERTICAL,598.0,8 1/2,1445.0,4
4,8030JC,1,VERTICAL,0.0,17 1/2,410.0,3


In [31]:
# Verificar tipos de cada coluna
print(df.dtypes)

nome_do_poco          object
fase                   int64
tipo_poco             object
lda_lamina_d_agua    float64
diametro_fase         object
metragem             float64
nfases                 int64
dtype: object


In [32]:
# Verificar quantidade de valores ausentes por coluna
missing_values = df.isna().sum()
missing_values_percentage = (missing_values / len(df)) * 100
missing_values_df = pd.DataFrame({'valores_nulos': missing_values, 'percentual': missing_values_percentage})
print(missing_values_df)

                   valores_nulos  percentual
nome_do_poco                   0         0.0
fase                           0         0.0
tipo_poco                      0         0.0
lda_lamina_d_agua              0         0.0
diametro_fase                  0         0.0
metragem                       0         0.0
nfases                         0         0.0


In [33]:
# Verificar quantidade de linhas duplicadas
duplicate_count = df.duplicated().sum()
print(f'Quantidade de linhas duplicadas: {duplicate_count}')

Quantidade de linhas duplicadas: 0


In [34]:
# Converte coluna de string para float
df['diametro_fase_polegadas'] = df['diametro_fase'].apply(polegadas)
df.drop(columns='diametro_fase', inplace=True)
df.dtypes

nome_do_poco                object
fase                         int64
tipo_poco                   object
lda_lamina_d_agua          float64
metragem                   float64
nfases                       int64
diametro_fase_polegadas    float64
dtype: object

## Corrigindo a coluna 'tipo_poco'
- O exercício informa que existem apenas 02 tipos: VERTICAL e HORIZONTAL
- Mas o dataset tem mais valores além desses: DIRECIONAL e '0'
- data imputting

In [35]:
df['tipo_poco'].unique()

array(['VERTICAL', 'DIRECIONAL', '0', 'HORIZONTAL'], dtype=object)

Diante da quantidade de valores incorretos, tenho que pensar numa abordagem cautelosa para substituir esses valores.

In [36]:
# Contando quantas linhas têm valores incorretos para 'tipo_poco'
counts = df['tipo_poco'].value_counts()
direcional_count = counts.get('DIRECIONAL', 0)
zero_count = counts.get('0', 0)

# Calcular a porcentagem
total_count = len(df['tipo_poco'])
direcional_perc = (direcional_count / total_count) * 100
zero_perc = (zero_count / total_count) * 100

print(f"Porcentagem de valores com DIRECIONAL: {direcional_perc:.2f}%")
print(f"Porcentagem de valores com 0: {zero_perc:.2f}%")

Porcentagem de valores com DIRECIONAL: 35.80%
Porcentagem de valores com 0: 1.42%


### Imputação de dados com Machine Learning
- Vou utilizar um modelo de Machine Learning para prever (**classificar**) o valor correto de 'tipo_poco' com base nas características das ocorrências corretas do dataset. 
- Esse modelo será aplicado às ocorrências com 'tipo_poco' = 'DIRECIONAL'
- **Por que escolhi essa abordagem?** Para preservar ao máximo as características do dataset após a imputação de dados. Usar um método mais simples (como a moda) sobre uma grande quantidade de ocorrências (~36%) poderia induzir os modelos de ML ao erro durante a fase de modelagem, pois ele poderia encontrar características de um poço HORIZONTAL junto com o 'tipo_poco' VERTICAL, por exemplo.

In [37]:
# Primeiro eu vou tratar as ocorrências com zero: vou remover todas as linhas com valores '0', já que são poucas (1.4%)
df = df[df['tipo_poco'] != '0']

In [44]:
# Definir os modelos e seus respectivos grids de parâmetros
model_param_grid = {
    'RandomForest': (RandomForestClassifier(), {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski']
    }),
    'DecisionTree': (DecisionTreeClassifier(), {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    })
}

# Testar os modelos e obter o melhor
best_model = seleciona_modelos(df, model_param_grid)


Testando RandomForest...



Testando KNN...

Testando DecisionTree...

Melhor Modelo:
Modelo: RandomForest
Melhores Parâmetros Encontrados:
{'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Relatório de Classificação (Treinamento):
              precision    recall  f1-score   support

  HORIZONTAL       0.96      0.81      0.88      1715
    VERTICAL       0.94      0.99      0.96      4971

    accuracy                           0.94      6686
   macro avg       0.95      0.90      0.92      6686
weighted avg       0.94      0.94      0.94      6686

Acurácia no Conjunto de Treinamento:
0.9429


In [46]:
# Gravar melhor modelo de imputação
joblib.dump(best_model, 'models/best_model.pkl')

['models/best_model.pkl']

In [58]:
# Imputar os dados corretos em 'tipo_poco' utilizando o melhor modelo de imputação

# Carregar o melhor modelo encontrado
best_model = joblib.load('models/best_model.pkl')

# Preparar os dados para previsão
df_direcional = df[df['tipo_poco'] == 'DIRECIONAL'] # monta um df apenas com as ocorrências de 'DIRECIONAL'
X_direcional = df_direcional[['fase', 'lda_lamina_d_agua', 'metragem', 'nfases', 'diametro_fase_polegadas']]

# Codificar o target apenas com os rótulos de 'HORIZONTAL' e 'VERTICAL'
label_encoder = LabelEncoder()
label_encoder.fit(['HORIZONTAL', 'VERTICAL'])  # Ajustar encoder com rótulos do modelo --> modelo foi treinado para retornar '0' e '1' apenas (HORIZONTAL e VERTICAL, não nessa ordem)

# Realizar previsões
y_pred_encoded = best_model.predict(X_direcional)

# Converter previsões de volta para os rótulos originais
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Substituir valores 'DIRECIONAL' com previsões
df.loc[df['tipo_poco'] == 'DIRECIONAL', 'tipo_poco'] = y_pred

In [65]:
# Gravar dataframe limpo
df.to_csv('data/df_cleaned.csv', index=False)
print('Dataframe limpo gravado com sucesso!')

Dataframe limpo gravado com sucesso!


# **Análise Exploratória de Dados (EDA)**
- Extrair insights de negócio
- Estatísticas descritivas
- Distribuições de variáveis
- Análise de correlação
- Visualizações (histogramas, scatter plots, heatmaps, etc.)

In [69]:
# Carregando dataset limpo
df = pd.read_csv('data/df_cleaned.csv')
df.head()

,nome_do_poco,fase,tipo_poco,lda_lamina_d_agua,metragem,nfases,diametro_fase_polegadas
0,6C5G4M,1,VERTICAL,598.0,607.0,4,26.00
1,6C5G4M,2,VERTICAL,598.0,1306.0,4,17.50
2,6C5G4M,3,VERTICAL,598.0,1201.0,4,12.25
3,6C5G4M,4,VERTICAL,598.0,1445.0,4,8.50
4,8030JC,1,VERTICAL,0.0,410.0,3,17.50


## Estatísticas Descritivas

In [70]:
# Informações gerais do dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10498 entries, 0 to 10497
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   nome_do_poco             10498 non-null  object 
 1   fase                     10498 non-null  int64  
 2   tipo_poco                10498 non-null  object 
 3   lda_lamina_d_agua        10498 non-null  float64
 4   metragem                 10498 non-null  float64
 5   nfases                   10498 non-null  int64  
 6   diametro_fase_polegadas  10498 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 574.2+ KB


In [71]:
# Estatística descritiva (apenas das colunas numéricas)
df.describe().round(2)

,fase,lda_lamina_d_agua,metragem,nfases,diametro_fase_polegadas
count,10498.00,10498.00,10498.00,10498.00,10498.00
mean,2.67,1115.23,680.76,4.27,16.80
std,1.54,729.06,563.40,1.81,8.54
min,1.00,0.00,0.00,1.00,0.00
25%,1.00,466.75,188.25,3.00,9.50
50%,2.00,1098.00,609.00,4.00,12.25
75%,4.00,1735.00,1027.00,5.00,22.00
max,8.00,2988.00,4040.00,9.00,42.00


## Análise da distribuição dos dados

In [75]:


# Analisar a distribuição de variáveis numéricas
numeric_columns = ['fase', 'lda_lamina_d_agua', 'metragem', 'nfases', 'diametro_fase_polegadas']
plot_distributions_grid(df, numeric_columns)


## Análise de correlações

## Análise Univariada

## Análise Bivariada

# **Feature Engineering**
- Criação de novas variáveis
- Seleção de variáveis
- Transformações de variáveis
- Codificação de variáveis categóricas

# **Pré-processamento dos Dados**
- Divisão dos dados em treino e teste
- Escalonamento de características
- Preparação de dados para modelagem

# **Modelagem** (Machine Learning)
- Escolha dos modelos
- Treinamento dos modelos
- Avaliação de desempenho dos modelos
- Ajuste de hiperparâmetros

# **Avaliação do Modelo**
- Métricas de avaliação (precision, recall, F1-score, etc.)
- Curvas ROC e AUC
- Análise de erros

# **Deploy do Modelo**
- Gravação do modelo treinado
- Carregamento do modelo para previsões futuras

# **Conclusões**
- Sumário dos resultados
- Recomendações
- Próximos passos

# **Referências**
- Citações e referências utilizadas